In [1]:
pip install folium

In [1]:
import pypsa
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import folium
import math
import networkx as nx

In [3]:
initial_network = pd.read_csv('database/lines_kalimantan.csv')

initial_network

,nodeA,nodeB,length_m
0,GI_Sambas,GI_Singkawang,59200
1,GI_Singkawang,GI_PLTU_2_Kalbar,20829
2,GI_PLTU_2_Kalbar,GI_PLTU_3_Kalbar,1393
3,GI_Singkawang,GI_Senggiring,70613
4,GI_Senggiring,GI_Parit_Baru,42848
5,GI_Parit_Baru,GI_Kota_Baru,22011
6,GI_Parit_Baru,GI_Siantan,16708
7,GI_Siantan,GI_Sei_Raya,18140
8,GI_Siantan,GI_Tayan,99173
9,GI_Tayan,GI_Ngabang,47271


In [4]:
initial_network_nodes = pd.read_csv('database/buses_substation_kalimantan.csv')

initial_network_nodes

,node,type,lon,lat,frequency,voltage
0,GI_Sambas,substation,109.2688,1.3429,50,150
1,GI_Singkawang,substation,108.9876,0.9266,50,150
2,GI_PLTU_3_Kalbar,substation,108.8747,0.8361,50,150
3,GI_PLTU_2_Kalbar,substation,108.8653,0.8300,50,150
4,GI_Senggiring,substation,108.9853,0.3162,50,150
5,GI_Parit_Baru,substation,109.2055,0.0592,50,150
6,GI_Kota_Baru,substation,109.2987,-0.0754,50,150
7,GI_Siantan,substation,109.3280,-0.0002,50,150
8,GI_Sei_Raya,substation,109.3809,-0.0735,50,150
9,GI_Tayan,substation,110.0932,0.0015,50,150


In [5]:
combined = initial_network.merge(initial_network_nodes, left_on = "nodeA", right_on = "node", suffixes = ("_left", "_right"))
combined = combined.merge(initial_network_nodes, left_on  = "nodeB", right_on = "node", suffixes = ("_A", "_B"))
combined = combined [["nodeA", "lon_A", "lat_A", "nodeB", "lon_B", "lat_B"]]

display(combined)

,nodeA,lon_A,lat_A,nodeB,lon_B,lat_B
0,GI_Sambas,109.2688,1.3429,GI_Singkawang,108.9876,0.9266
1,GI_Bengkayang,109.4967,0.9056,GI_Singkawang,108.9876,0.9266
2,GI_Singkawang,108.9876,0.9266,GI_PLTU_2_Kalbar,108.8653,0.8300
3,GI_Singkawang,108.9876,0.9266,GI_Senggiring,108.9853,0.3162
4,GI_PLTU_2_Kalbar,108.8653,0.8300,GI_PLTU_3_Kalbar,108.8747,0.8361
5,GI_Senggiring,108.9853,0.3162,GI_Parit_Baru,109.2055,0.0592
6,GI_Parit_Baru,109.2055,0.0592,GI_Kota_Baru,109.2987,-0.0754
7,GI_Parit_Baru,109.2055,0.0592,GI_Siantan,109.3280,-0.0002
8,GI_Siantan,109.3280,-0.0002,GI_Sei_Raya,109.3809,-0.0735
9,GI_Siantan,109.3280,-0.0002,GI_Tayan,110.0932,0.0015


In [11]:
basemap = folium.Map(location = [1.069424, 109.972163], zoom_start = 8)

maxvoltage = max(initial_network_nodes.voltage)

for i in range(len(initial_network_nodes)):
    popupinfo = (initial_network_nodes.node.iloc[i]+ ",voltage:" + str(initial_network_nodes.voltage.iloc[i]))
    folium.CircleMarker(location = [initial_network_nodes.lat.iloc[i], initial_network_nodes.lon.iloc[i]], 
                       tooltip = popupinfo, color = "green", fill = True, 
                       radius = math.sqrt(100 * initial_network_nodes.voltage.iloc[i] / maxvoltage)).add_to(basemap)

for i in range(len(combined)):
    points = [[combined.lat_A.iloc[i], combined.lon_A.iloc[i]],
             [combined.lat_B.iloc[i], combined.lon_B.iloc[i]]]
    folium.PolyLine(points, color = "yellow", opacitiy = 0.7).add_to(basemap)
    
basemap.save("initial_nodes.html")

display(basemap)